In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, LSTM, Dense
import matplotlib.pyplot as plt

In [6]:
df_labels = pd.read_csv('keyframe_features_dataset.csv')
window_map = {'MaxKneeLift': 2}
expanded_rows = []
for _, row in df_labels.iterrows():
    w = window_map.get(row['Keyframe_Type'], 1)
    for off in range(-w, w+1):
        fn = row['Frame'] + off
        if fn >= 0:
            expanded_rows.append({
                'Video': row['Video'],
                'Frame': fn,
                'Keyframe_Type': row['Keyframe_Type']
            })
df_labels_expanded = pd.DataFrame(expanded_rows)
print("Expanded label counts:\n", df_labels_expanded['Keyframe_Type'].value_counts())


Expanded label counts:
 Keyframe_Type
MaxKneeLift              135
LiftOff                   90
DragStart                 84
CirclePeak                84
FootPlant                 84
Connection                84
Release                   84
ThreeQuarterArmCircle     78
Name: count, dtype: int64


In [7]:
df_feat = pd.read_csv('keyframe_features_dataset.csv')
# Build lookup
feat_dict = {
    (r['Video'], r['Frame']): r[['Elbow_Angle','Trunk_Lean_Angle','Stride_Length',
           'Humerus_Straightness','UpperArm_Compression','Hand_Center_Offset']].values
    for _, r in df_feat.iterrows()
}

W = 2
X_seqs, y_labels = [], []
for _, row in df_labels_expanded.iterrows():
    seq = []
    ok = True
    for off in range(-W, W+1):
        key = (row['Video'], row['Frame'] + off)
        feat = feat_dict.get(key)
        if feat is None:
            ok = False
            break
        seq.append(feat)
    if not ok:
        continue
    X_seqs.append(seq)
    y_labels.append(row['Keyframe_Type'])
X_seqs = np.array(X_seqs)
y_labels = np.array(y_labels)
print("Sequence data shape:", X_seqs.shape)


Sequence data shape: (0,)


In [4]:
le = LabelEncoder()
y_enc = le.fit_transform(y_labels)
print("Classes:", le.classes_)

X_train, X_test, y_train, y_test = train_test_split(
    X_seqs, y_enc, test_size=0.2, stratify=y_enc, random_state=42
)
print("Train/Test shapes:", X_train.shape, X_test.shape)


Classes: []


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
n_features = X_seqs.shape[2]
n_classes  = len(le.classes_)

model = Sequential([
    Masking(mask_value=0.0, input_shape=(2*W+1, n_features)),
    LSTM(32),
    Dense(n_classes, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20, batch_size=16
)

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()


In [ ]:
y_pred = model.predict(X_test).argmax(axis=1)
print(classification_report(y_test, y_pred, target_names=le.classes_))
